In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
model=load_model('model.h5')

#Load encoder,scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)
with open('ohe_geo.pkl','rb') as file:
    ohe_geo = pickle.load(file)
with open('myscaler.pkl','rb') as file:
    scaler = pickle.load(file)

#Dont worry abou the warning

In [3]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
df=pd.DataFrame([input_data])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [5]:
# One Hot Encode Geography
geo_encoded = ohe_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=ohe_geo.get_feature_names_out(['Geography']))
df = pd.concat([(df.drop('Geography',axis=1)),geo_encoded_df],axis=1)
df

c:\Users\saivi\OneDrive\Documents\coding\python\jupyter_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [6]:
df['Gender'] = label_encoder_gender.transform(df['Gender'])
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [7]:
df_scaled=scaler.transform(df)
df_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [8]:
# Prediction
prediction = model.predict(df_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step


array([[0.04714475]], dtype=float32)

In [9]:
probability = prediction[0][0]
if probability>0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
